# 샘플러 검증

In [19]:
import json
def get_dataset(self, labels_file_path, data_file_path):
        with open(labels_file_path, "r") as file:
            labels = json.load(file)
            labels = labels["categories"]

        with open(data_file_path, "r") as file:
            data = json.load(file)
            data = data["images"]

        return labels, data

In [25]:
train_file_name = "../data/aihub_1.0_43_0.3_train_crop.json"
test_file_name = "../data/aihub_1.0_43_0.3_test_crop.json"

with open(train_file_name, "r") as file:
    data = json.load(file)
    data = data["images"]

In [29]:
labels_file_path = "../data/labels.json"

with open(labels_file_path, "r") as file:
    labels = json.load(file)
    labels = labels["categories"]

In [36]:
def get_id_to_text(label_data):
        return {item["id"]: item["label"] for item in label_data}

def get_text_to_id(label_data):
    return {item["label"]: item["id"] for item in label_data}

id_to_text_dict = get_id_to_text(labels)
text_to_id_dict = get_text_to_id(labels)

In [38]:
cls = {}
for ind, d in enumerate(data):
    label = d["category_id"]
    if label in cls:
        cls[label].append(ind)
    else:
        cls[label] = [ind]

cls_inds = [0 for _ in range(len(cls))]

cls_indicies = list(cls.keys())
cls_matcher = {idx: idx for idx in range(len(cls_inds))}

indicies = []
ind_step = cls_indicies[0]
for _ in range(len(data)):
    cls_ind = cls_indicies[ind_step % len(cls)]
    cls_ind = cls_matcher[cls_ind]
    smp_ind = cls_inds[cls_ind] % len(cls[cls_ind])
    index = cls[cls_ind][smp_ind]

    ind_step += 1
    cls_inds[cls_ind] += 1
    indicies.append((index,id_to_text_dict[cls_ind]))

In [2]:
import argparse
import json

import torch

from src.dataset import FoodImageDataset
from src.model import build_model
from src.preprocess import image_transform
from src.tokenizer import get_tokenizer

with open("src/model_configs/baseline.json") as f:
    configs = json.load(f)
vision_cfg = configs["vision_cfg"]

preprocess = image_transform(vision_cfg["image_size"], is_train=True)
dataset = FoodImageDataset(preprocess, mode="train")

FileNotFoundError: [Errno 2] No such file or directory: 'data/labels.json'